In [1]:
import findspark

In [2]:
# COMMAND ----------
# Databricks notebook source
from multiprocessing import current_process
import glob
import pyspark.pandas as pf
import pandas as pd
from pathlib import Path
import warnings
import logging
import sys
import os
import time
import pyspark
from delta import *
from pyspark.conf import SparkConf

conf = SparkConf()
conf.set("spark.driver.maxResultSize", "8g")
conf.set("spark.driver.memory", "8g") 
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")


builder = pyspark.sql.SparkSession.builder.appName("dev_test_notebook") \
    .enableHiveSupport()\
    .config(conf=conf) 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb4b6c0f-ddc4-42ee-b9ef-f9a9887ddb94;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 163ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

23/08/16 09:34:33 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/08/16 09:34:33 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/08/16 09:34:33 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


In [ ]:
import pandas as pd

file_path = 'file:/landing/Raw/Merlin/ecb2ef6fdf3042339961920fee90b950/ecb2ef6fdf3042339961920fee90b950/DIS_NEST_NEST_00000300_0.parquet'
read_local_file_pd = pd.read_parquet(file_path, engine='fastparquet')
read_local_file_pd.head(10)

In [ ]:
read_local_file_pf = pd.read_parquet(file_path, engine='fastparquet')
pf_df = pf.DataFrame(read_local_file_pf)
pf_df.head(10)


In [ ]:
# Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)) error en los parquet
read_local_file_pf = pf.read_parquet(file_path,engine='fastparquet')
read_local_file_pf.head(10)

In [ ]:
# read file in the hdbs if is local file:/
file_path = 'file:/landing/Staging/Merlin/Smartquoting/3cb9a027-3b3e-4f3d-b9df-d8478cf812c8/8abe3f95-37f1-4cbc-a152-52e79ed26a65/staging_db_data_production.csv'
read_local_file_pf = pf.read_csv(file_path,sep=";",
                         decimal=',',
                         encoding='ISO-8859-1',
                         quotechar='"',
                         engine='python',
                         dtype=str,
                         na_values='nan',
                         inferSchema=True)


In [ ]:
read_local_file_pf = read_local_file_pf[["Part_IsAssembly", "Part_Weight", "Part_RealArea", "Part_ExternalArea", "Part_Thickness", "Part_Length", "Part_RectangularArea",
            "Part_IsFillerPart", "Part_MaterialDensity", "Part_RectangularWeight", "Part_MarkingPerimeter","Part_ExternalWeight", "Part_CutPerimeter",
            "Part_Width", "ManufacturingOrder_Quantity", "Part_WorkCenter", "Part_Material", "Part_Rotations", "Part_CreationMethod", "Tenant_ID",
            "Plant_ID"]]
read_local_file_spf = read_local_file_pf.to_spark()
display(read_local_file_spf.head(10))

In [ ]:
# Write the DataFrame to a Delta table in Hive
hive_table_name = 'default.cr_table_demo_delta'
hive_table_path= f'/user/jupyter/result/{hive_table_name}'

# write table 
read_local_file_spf.write.format("delta").mode('overwrite').save(hive_table_path)



In [ ]:
# Get the schema of the DataFrame
schema = read_local_file_spf.schema
# Create the table in Hive
column_definitions = ", ".join([f"{field.name} {field.dataType.simpleString()}" for field in schema])
print(column_definitions)
# Create an external Hive table pointing to the Delta table location
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {hive_table_name} ({column_definitions}) USING delta LOCATION '{hive_table_path}'")

In [ ]:
# Create an external Hive table pointing to the Delta table location
hive_table_name = 'default.cr_table_demo_hive'
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {hive_table_name} ({column_definitions}) USING hive LOCATION '{hive_table_path}'")

In [3]:
hive_table_name = 'default.sklearn_housing'
len(spark.sql(f"select * from {hive_table_name}").tail(10))

23/08/16 09:34:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


10

In [4]:
spark.stop()